# Arguments

In [1]:

warmup_model_path = 'model-warmup.h5'
model_save_path = 'model-final.h5'
dataset_name = 'tomato-dataset'
dataset_path = '../datasets/' + dataset_name
checkpoint_path = 'checkpoints-finetuning.hdf5'
input_width = 299
input_height = 299
input_depth = 3

batch_size = 32
num_of_epochs = 100

# Get classes
import os
import re
classes = os.listdir(dataset_path)
class_names = []

for i in classes:
    if(re.search("Tomato___", i)):
        class_names.append(i)
    
print('Classes: ', class_names)
print(len(class_names))


Classes:  ['Tomato___Tomato_mosaic_virus', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Bacterial_spot', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___healthy', 'Tomato___Septoria_leaf_spot', 'Tomato___Two-spotted_spider_mite']
10


# Load model from disk

In [2]:
# import tensorflow as tf
# print(tf.__version__)


# model = tf.keras.models.load_model(
#     warmup_model_path,
#     custom_objects=None,
#     compile=False
# )

from keras.models import load_model

model = load_model(warmup_model_path)

Using TensorFlow backend.


# Print index of layers

In [3]:
# iterate for all layers in the network and print its' index value
# for (i,layer) in enumerate(model.layers):
#     print("[INFO] {}\t{}".format(i,layer.__class__.__name__))

# Unfreeze final CONV layers

In [4]:
for layer in model.layers[294:]:
    layer.trainable = True
print('unfrozen')

unfrozen


# Compile model

In [5]:
from keras.optimizers import SGD

# Build the model from the new
print("[INFO] re-compiling model ...")
opt = SGD(lr=0.001)

# Fine-tuning with a small learning rate
model.compile(loss = 'categorical_crossentropy',optimizer = opt,
              metrics=['accuracy'])

[INFO] re-compiling model ...


# Load data

In [6]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image   import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

import sys
sys.path.append('..')

from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader

# construct the image generator for data augmentation
aug = ImageDataGenerator(   rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

# Load image paths
print("[INFO] loading images...")
image_paths = list(paths.list_images(dataset_path))

# Initial image preprocessing
aap = AspectAwarePreprocessor(input_width, input_height)
iap= ImageToArrayPreprocessor()

#Load image data and perform image data preprocessing
sdl = SimpleDatasetLoader(preprocessors=[aap,iap])
(data,labels)  = sdl.load(image_paths,verbose=500)
data = data.astype("float") / 255.0


# train test split
(train_x,test_x,train_y,test_y) = train_test_split(data,labels,test_size=0.25,random_state=42)

# convert the labels from integers to vectors
train_y = LabelBinarizer().fit_transform(train_y)
test_y = LabelBinarizer().fit_transform(test_y)

[INFO] loading images...
[INFO]: Processed 500/18160
[INFO]: Processed 1000/18160
[INFO]: Processed 1500/18160
[INFO]: Processed 2000/18160
[INFO]: Processed 2500/18160
[INFO]: Processed 3000/18160
[INFO]: Processed 3500/18160
[INFO]: Processed 4000/18160
[INFO]: Processed 4500/18160
[INFO]: Processed 5000/18160
[INFO]: Processed 5500/18160
[INFO]: Processed 6000/18160
[INFO]: Processed 6500/18160
[INFO]: Processed 7000/18160
[INFO]: Processed 7500/18160
[INFO]: Processed 8000/18160
[INFO]: Processed 8500/18160
[INFO]: Processed 9000/18160
[INFO]: Processed 9500/18160
[INFO]: Processed 10000/18160
[INFO]: Processed 10500/18160
[INFO]: Processed 11000/18160
[INFO]: Processed 11500/18160
[INFO]: Processed 12000/18160
[INFO]: Processed 12500/18160
[INFO]: Processed 13000/18160
[INFO]: Processed 13500/18160
[INFO]: Processed 14000/18160
[INFO]: Processed 14500/18160
[INFO]: Processed 15000/18160
[INFO]: Processed 15500/18160
[INFO]: Processed 16000/18160
[INFO]: Processed 16500/18160
[INFO

# Checkpoints

In [7]:
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', mode='max', 
save_best_only=True, verbose=1)

callbacks = [checkpoint]

#load checkpoints if existing
import os

epochs_done = 0
num_of_epochs = 1


if(os.path.exists(checkpoint_path)):
    model.load_weights(checkpoint_path)
    num_of_epochs = num_of_epochs - epochs_done
    print('checkpoints loaded')

checkpoints loaded


# Training Monitor

In [8]:
from utils.callbacks.training_monitor import TrainingMonitor
import pathlib

# Construct the set of callbacks
# fig_path = os.path.sep.join([str(pathlib.Path().absolute()), "/{}.png".format(os.getpid())])
# json_path = os.path.sep.join([str(pathlib.Path().absolute()), "/{}.json".format(os.getpid())])

# reload
start_epoch = 0
fig_path = "3570" + ".png"
json_path = "3570" + ".json"

callbacks.append(TrainingMonitor(fig_path, json_path, start_epoch))

# Fit model

In [9]:
H = model.fit_generator(
    aug.flow(train_x,train_y, batch_size = 32),
             validation_data = (test_x,test_y),
             epochs=num_of_epochs,
             steps_per_epoch = len(train_x) //32,
             verbose = 1,
             callbacks=callbacks)

Epoch 1/1
424/425 [============================>.] - ETA: 0s - loss: 1.6275 - accuracy: 0.3933

KeyboardInterrupt: 

In [ ]:
# from keras.models import load_model
# import os

# model = load_model(warmup_model_path)


# if(os.path.exists(checkpoint_path)):
#     model.load_weights(checkpoint_path)
#     print('weights loaded')

# Save model

In [ ]:
model.save(model_save_path)

# Reload model from last best checkpoint

In [ ]:
# from keras.models import load_model
# import os

# model = load_model(warmup_model_path)


# if(os.path.exists(checkpoint_path)):
#     model.load_weights(checkpoint_path)
#     print('weights loaded')

# Load model form disk

In [ ]:

# # from keras.models import load_model

# # model_save_path = 'model-final.h5'
# # model = load_model(model_save_path)

# Evaluate

In [ ]:
from sklearn.metrics import classification_report

print("[INFO] evaluating after initialization...")
predictions = model.predict(test_x,batch_size=batch_size)

print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),
                            target_names=class_names, 
                            digits=4))

# Visualize

In [ ]:
plot_save_path = 'plot-fine-tuning.png'

# plot the training loss and accuracy
from matplotlib import pyplot as plt

plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, num_of_epochs), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, num_of_epochs), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, num_of_epochs), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, num_of_epochs), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
#save to disk
plt.savefig(plot_save_path)
plt.show()